In [1]:
import ROOT as rt

Welcome to JupyROOT 6.14/04


In [2]:
def readdir(file, outfile, inpath = "", outpath = "", new_hist_postfix = "", target_postfix = "") :
    print("readir called....")
    dirSave = file.GetDirectory(inpath)
    keys = dirSave.GetListOfKeys() 
    
    for key in keys :
        if key.IsFolder() :
            print("folder: " , key.GetName())
            if inpath == "" :
                out_dir = key.GetName()
                
                if new_hist_postfix != "" and new_hist_postfix in out_dir :
                    temp_list = out_dir.split("_")
                    temp_list.remove(new_hist_postfix)
                    out_dir = "_".join(temp_list) 
                
                print("creat a directory... ", out_dir)
                outfile.mkdir(out_dir)
                readdir(file, outfile, 
                        key.GetName() + "/", out_dir + "/", 
                        new_hist_postfix, target_postfix)
            else :
                
                out_dir = key.GetName()
                
                if new_hist_postfix != "" and new_hist_postfix in out_dir :
                    out_dir = "_".join(out_dir.split("_")[:-1]) #
                    
                print("creat a directory... ", outpath + out_dir + "/")
                outfile.mkdir(outpath + out_dir + "/")
                readdir(file, outfile, 
                        inpath + key.GetName() + "/", outpath + out_dir + "/", 
                        new_hist_postfix, target_postfix)
            continue
            
        temp_obj = file.Get(inpath + "/" + key.GetName()) 
        
        if type(temp_obj) == rt.TH1D or type(temp_obj) == rt.TH2D:
            if new_hist_postfix != "" :
                print("Histogram ", key.GetName(), type(temp_obj))
                temp_obj.SetName(key.GetName() + "_" + new_hist_postfix)
                outfile.cd(outpath)
                temp_obj.Write()
            else :
                if target_postfix in key.GetName() :
                    outfile.cd(outpath)
                    temp_obj.Write()
    

In [7]:
# For lepton momentum correction
dir1_list = ["LepMomScale", "LepMomRes"]
dir2_list = ["Up", "Down"]

for dir1 in dir1_list :
    for dir2 in dir2_list :
        inputHistFilePath = "./inFiles/2017/electron_extended/" + dir1 + "/" + dir2 + "/"
        file=rt.TFile.Open(inputHistFilePath + "unfold_input.root", 'READ')
        ofile = rt.TFile.Open(inputHistFilePath + "unfold_input_new.root", "RECREATE")
        readdir(file, ofile, "", "", dir1 + dir2)
        ofile.Close()

readir called....
folder:  Detector_LepMomScaleUp
creat a directory...  Detector
readir called....
folder:  SubLeading_Lep_Eta_4
creat a directory...  Detector/SubLeading_Lep_Eta_4/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToTauTau <class 'ROOT.TH1D'>
Histogram  histo_DoubleEG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToEE <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToEE <class 'ROOT.TH1D'>
Histogram  histo_WZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_WW_pythia <class 'ROOT.TH1D'>
folder:  SubLeading_Lep_Eta_5
creat a directory...  Detector/SubLeading_Lep_Eta_5/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histo

readir called....
folder:  Detector_LepMomScaleDown
creat a directory...  Detector
readir called....
folder:  SubLeading_Lep_Eta_4
creat a directory...  Detector/SubLeading_Lep_Eta_4/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToTauTau <class 'ROOT.TH1D'>
Histogram  histo_DoubleEG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToEE <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToEE <class 'ROOT.TH1D'>
Histogram  histo_WZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_WW_pythia <class 'ROOT.TH1D'>
folder:  SubLeading_Lep_Eta_5
creat a directory...  Detector/SubLeading_Lep_Eta_5/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
His

readir called....
folder:  Detector_LepMomResUp
creat a directory...  Detector
readir called....
folder:  SubLeading_Lep_Eta_4
creat a directory...  Detector/SubLeading_Lep_Eta_4/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToTauTau <class 'ROOT.TH1D'>
Histogram  histo_DoubleEG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToEE <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToEE <class 'ROOT.TH1D'>
Histogram  histo_WZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_WW_pythia <class 'ROOT.TH1D'>
folder:  SubLeading_Lep_Eta_5
creat a directory...  Detector/SubLeading_Lep_Eta_5/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histogr

readir called....
folder:  Detector_LepMomResDown
creat a directory...  Detector
readir called....
folder:  SubLeading_Lep_Eta_4
creat a directory...  Detector/SubLeading_Lep_Eta_4/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToTauTau <class 'ROOT.TH1D'>
Histogram  histo_DoubleEG <class 'ROOT.TH1D'>
Histogram  histo_DYJetsToEE <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToEE <class 'ROOT.TH1D'>
Histogram  histo_WZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_WW_pythia <class 'ROOT.TH1D'>
folder:  SubLeading_Lep_Eta_5
creat a directory...  Detector/SubLeading_Lep_Eta_5/
readir called....
Histogram  histo_ZZ_pythia <class 'ROOT.TH1D'>
Histogram  histo_TTLL_powheg <class 'ROOT.TH1D'>
Histogram  histo_DYJets10to50ToTauTau <class 'ROOT.TH1D'>
Histogram  histo_WJets_MG <class 'ROOT.TH1D'>
Histo

In [ ]:

inputHistFilePath = "./inFiles/2016/electron/pdf/"
file = rt.TFile.Open(inputHistFilePath + "unfold_input_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "unfold_input_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()

In [ ]:

inputHistFilePath = "./inFiles/2016/electron/pdf/"
file = rt.TFile.Open(inputHistFilePath + "DY_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "DY_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()


In [ ]:
inputHistFilePath = "./inFiles/2016/electron/pdf/"
file = rt.TFile.Open(inputHistFilePath + "DY_FSR_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "DY_FSR_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()


In [ ]:
# For lepton momentum correction
dir1_list = ["LepScale", "LepRes"]
dir2_list = ["Up", "Down"]

for dir1 in dir1_list :
    for dir2 in dir2_list :
        inputHistFilePath = "./inFiles/2016/electron/" + dir1 + "/" + dir2 + "/"
        file=rt.TFile.Open(inputHistFilePath + "DY.root", 'READ')
        ofile = rt.TFile.Open(inputHistFilePath + "DY_new.root", "RECREATE")
        readdir(file, ofile, "", "", dir1 + dir2)
        ofile.Close()